In [23]:
def MeanECGExtraction(x, phase, bins, flag):

    meanPhase = np.zeros(bins)
    ECGmean = np.zeros(bins)
    ECGmedian = np.zeros(bins)
    ECGsd = np.zeros(bins)
    SamplesPerBin = np.zeros(bins)

    I = np.nonzero(np.logical_or(phase >= (np.pi - np.pi/bins), 
                                 phase < (-np.pi+np.pi/bins)))[0].tolist()
    #print('I is :', I,'\n')
    SamplesPerBin[0] = len(I)
    if I:
        meanPhase[0] = -np.pi
        ECGmean[0] = np.mean([x[i] for i in I])                        
        ECGmedian[0] = np.median([x[i] for i in I])
        ECGsd[0] = np.std([x[i] for i in I])
    else:
        meanPhase[0] = 0
        ECGmean[0] = 0
        ECGmedian[0] = 0
        ECGsd[0] = -1
    for i in np.arange(bins):
        I = np.nonzero(np.logical_and(phase >= 2*np.pi*(i-0.5)/bins-np.pi, 
                                      phase < 2*np.pi*(i+0.5)/bins-np.pi))[0].tolist()            
        SamplesPerBin[i] = len(I);
        if I:
            meanPhase[i] = np.mean([phase[i] for i in I])
            ECGmean[i] = np.mean([x[i] for i in I])
            ECGmedian[i] = np.median([x[i] for i in I])
            ECGsd[i] = np.std([x[i] for i in I])
        else:
            meanPhase[i] = 0
            ECGmean[i] = 0
            ECGmedian[i] = 0
            ECGsd[i] = -1
    K = np.nonzero(ECGsd == -1)[0].tolist() 
    #print('K is :', K, '\n')
    for i in np.arange(len(K)):
        if K[i] == 1:
            meanPhase[K[i]] = -np.pi;
            ECGmean[K[i]] = ECGmean[K[i] + 1]
            ECGmedian[K[i]] = ECGmedian[K[i] + 1]
            ECGsd[K[i]] = ECGsd[K[i] + 1]
            SamplesPerBin[K[i]] = SamplesPerBin[K[i] + 1]
        elif K[i] == bins:
            meanPhase[K[i]] = np.pi;
            ECGmean[K[i]] = ECGmean[K[i] - 1]
            ECGmedian[K[i]] = ECGmedian[K[i] - 1]
            ECGsd[K[i]] = ECGsd[K[i] - 1]
            SamplesPerBin[K[i]] = SamplesPerBin[K[i] - 1]
        else:
            meanPhase[K[i]] = np.mean([meanPhase[K[i] - 1], meanPhase[K[i] + 1]])
            ECGmean[K[i]] = np.mean([ECGmean[K[i] - 1], ECGmean[K[i] + 1]])
            ECGmedian[K[i]] = np.median([ECGmedian[K[i] - 1], ECGmedian[K[i] + 1]])
            ECGsd[K[i]] = np.mean([ECGsd[K[i]-1], ECGsd[K[i] + 1]])
            SamplesPerBin[K[i]] = SamplesPerBin[K[i] - 1] + SamplesPerBin[K[i] + 1]
    if flag == 1:
        ECGmean = ECGmean - np.mean(ECGmean[1:np.ceil(len(ECGmean)/10)])
        ECGmedian = ECGmedian - np.median(ECGmedian[1:ceil(len(ECGmedian)/10)])

    return ECGmean.tolist(), ECGsd.tolist(), meanPhase.tolist(), ECGmedian.tolist(), SamplesPerBin.tolist()